# NER - NLTK

In [13]:
import pandas as pd

df = pd.read_csv('dataset.csv', encoding='utf-8')

#Remplazamos los tags que empiezan con 'O-' por 'I-'
df['Tag'] = df['Tag'].replace('O-','I-', regex=True)

In [14]:
df.head()

,Chat #,Sentence #,Word,Tag
0,1,0,Cliente,O
1,1,0,:,O
2,1,0,Hola,O
3,1,0,",",O
4,1,0,¿,O


### Etiquetadores para el modelo NER:

Los etiquetadores n-gramas calculas la probabilidad de que una palabra sea una de una categoría a partir de la productoria de las probabilidades de que una palabra sea de una categoría viendo sus n palabras anteriores.

Para el modelo reamos dos etiquetadores de trigramas:
- El primero va a ser entrenado con el corpus importado en español, el cual se va a encargar de reconocer y etiquetar las partes del lenguaje, ya que van a ser características importantes de cada palabra a la hora de reconocer la entidad. \
- El segundo etiquetador va a ser entrenado con el conjunto datos de entrenamiento para servir como refuerzo al clasificador de entidades.

In [ ]:
from nltk.corpus import cess_esp as cess
from nltk import UnigramTagger as ut
from nltk import BigramTagger as bt
from nltk import TrigramTagger as tt

In [17]:
#Separamos los datos en datos de entrenamiento y datos de testeo
labeled_text = []
pos_tagged_text = []

#Recorremos el dataframe agrupando por oraciones y chats
for _, sent in df.groupby(['Chat #', 'Sentence #']):
    sentence = []
    tokens = sent.Word.values.tolist()

    #Descartamos los dialogos de los empleados
    if tokens[0] != 'Empleado':
        for i,(_,word) in enumerate(sent.iterrows()):
            sentence.append((word['Word'], word['Tag']))        
        labeled_text.append(sentence)

#Asignamos el tamaño de entrenamiento y test
train_size = int(len(labeled_text)*80/100)

train_set = labeled_text[:train_size]
test_set = labeled_text[train_size:]

#Sacamos las etiquetas de las tuplas para nuestro set de test
test_unlabeled = [[tuple[0] for tuple in sublist] for sublist in test_set]

In [16]:
#Oraciones taggeadas del corpus
cess_sents = cess.tagged_sents()

#Entrenamos un etiquetador trigrama con el corpus cess para las POS tags
uni_tag = ut(cess_sents)
bi_tag = bt(cess_sents, backoff=uni_tag)
tri_tag = tt(cess_sents, backoff=bi_tag)

In [18]:
#Creamos un TrigramTagger para etiquetas las entidades
uni_tagger = ut(train_set)
bi_tagger = bt(train_set, backoff=uni_tagger)
tri_tagger = tt(train_set, backoff=bi_tagger)

### Modelo NER:

El modelo fue creado en base a un *NaiveBayesClassifier*. 

Este clasificador, fundamentado enel teorema de Bayes, calcula la probabilidad de que una secuencia sea de una determinada categoría, através de la productoria logaritmica de la probabilidades de que los elementos de dicha secuencia sean de tal categoría. A su vez, para entrenar el clasificador, le asignamos a cada *palabra* los siguientes *features*:

- **word:** La palabra actual

- **pos:** La parte del lenguaje a la que corresponde la palabra en la sentencia, el cual fue calculado apartir de un modelo de etiquetación por trigramas entrenado con cess esp, un corpus en español provisto por NLTK

- **shape:** Determina el formato de la palabra. Por ejemplo, si la palabra esta compuesta por númerosy su longitud es mayor a 8, entonces se le asigna *number* ; en caso de ser menor a 8, se le asigna *short_number* ; o en caso de poseer un @ como carácter, se le asigna *email*

In [19]:
import re

from nltk.tag.sequential import ClassifierBasedTagger
from nltk.classify import NaiveBayesClassifier

def featurizer(tokens, index, history):
    """
    Recibe una lista de tokens, el índice de la palabra a etiquetar y devuelve las características de la palabra
    """
    word = tokens[index]
    pos = tri_tag.tag(tokens)

    #Extraemos la forma de la palabra
    if re.match(r"[0-9]+(\.[0-9]*)?|[0-9]*\.[0-9]+$", word) and len(word) < 8:
        shape = "short_number"
    elif re.match(r"[0-9]+(\.[0-9]*)?|[0-9]*\.[0-9]+$", word) and len(word) >= 8:
        shape = "number"
    elif re.match(r"\W+$", word):
        shape = "punct"
    elif re.match("[A-Z][a-z]+$", word):
        shape = "upcase"
    elif re.match("[a-z]+$", word):
        shape = "downcase"
    elif re.match("[\w\.-]+@[\w\.-]+(\.[\w]+)+", word):
        shape = "email"
    elif re.match(r"\w+$", word):
        shape = "mixedcase"
    else:
        shape = "other"

    features = {
        'word' : word.lower(),
        'pos' : pos[index][1],
        'pos_1' : pos[index-1][1],
        'shape' : shape
    }

    return features

#Entrenamos el etiquetador, con un clasificador Naive Bayes, la función para detectar características y un TrigramTagger como refuerzo
ner_classifier = ClassifierBasedTagger(train = train_set, backoff=tri_tagger, classifier_builder=NaiveBayesClassifier.train, feature_detector=featurizer)

In [20]:
def ner_tagger(sentence):
    """
    Recibe una oración y devuelve un dataframe con las palabras y sus etiquetas
    """
    tagged = ner_classifier.tag(sentence.split())
    return pd.DataFrame(tagged, columns = ['Word', 'Tag'])

In [21]:
def ner_file_tagger(path):
    """
    Recibe una ruta de un archivo .txt y devuelve una lista con sus oraciones etiquetadas
    """
    with open(path, 'r', encoding='utf-8') as file:
        tagged_sents = []
        for line in file.readline():
            tagged_sents.append(ner_tagger(line))
    return tagged_sents

### Resultados:

Los resultados del modelo son variados, viendo las métricas se ve facilmente que entidades como *B-intencion* no fueron del todo bien reconocidas. Mientras que otras como *B-email*, *B-nombre* e *I-fecha-de-nacimiento* obtuvieron buenos resultados. 

A priori, es necesario un conjunto de datos de mayor magnitud para mejorar el rendimiento del modelo. La cantidad de ocurrencias de cada entidad en el dataset es pequeña, lo cual genera una gran complejidad a la hora de reconocerlas, como es en el caso de *B-fecha-de-ingreso* donde no se ha podido y además se confunde facilmente con *B-fecha-de-nacimiento*.

In [24]:
from sklearn.metrics import classification_report

y_test = sum(test_set, [])

y_pred = []

for sent in test_unlabeled:
    y_pred.append(ner_classifier.tag(sent))

y_pred = sum(y_pred, [])

df_test = pd.DataFrame(y_test, columns=['Word','Tag'])
df_pred = pd.DataFrame(y_pred, columns=['Word','Tag'])

print(classification_report(df_test['Tag'], df_pred['Tag'], zero_division=0))

                    precision    recall  f1-score   support

           B-email       1.00      1.00      1.00         5
         B-empresa       0.40      1.00      0.57         2
   B-fecha-ingreso       0.00      0.00      0.00         1
B-fecha-nacimiento       0.50      0.60      0.55         5
       B-intencion       0.10      0.40      0.15         5
          B-nombre       0.71      1.00      0.83         5
 B-numero-empleado       0.00      0.00      0.00         2
        B-telefono       0.43      1.00      0.60         3
I-fecha-nacimiento       0.75      1.00      0.86        12
       I-intencion       0.29      0.36      0.32        11
          I-nombre       0.45      1.00      0.62         5
                 O       0.99      0.95      0.97       978

          accuracy                           0.94      1034
         macro avg       0.47      0.69      0.54      1034
      weighted avg       0.96      0.94      0.95      1034



In [22]:
ner_tagger("Me llamo Teo Perez, nací el 8 de agosto de 1986 y vivo en la calle Moreno 456. Mi correo es carlosrodriguez@ibm.com y mi teléfono es el 022165432109. Trabajo en IBM desde el 2010. Quiero dar de baja un producto")

,Word,Tag
0,Me,O
1,llamo,O
2,Teo,B-nombre
3,"Perez,",I-nombre
4,nací,O
5,el,O
6,8,B-fecha-nacimiento
7,de,I-fecha-nacimiento
8,agosto,I-fecha-nacimiento
9,de,I-fecha-nacimiento


In [23]:

pd.DataFrame(ner_classifier.tag(test_unlabeled[4]),columns=['Word','Tag'])


,Word,Tag
0,Cliente,O
1,:,O
2,Hola,O
3,",",O
4,soy,O
5,Martín,B-nombre
6,Rodríguez,I-nombre
7,y,O
8,los,O
9,llamo,O
